In [33]:
"""
DSC540 Week 7 and 8 assignment Exercise
"""

from __future__ import print_function
from itertools import zip_longest

import csv
import logging
import sys
import numpy as np
import pandas as pd
import random
import thinkplot
import thinkstats2
import datetime
import regression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
%matplotlib inline

def ReadData(filename):
    # File was throwing some encoding error while trying to read with default utf-8 encoding
    # Using latin-1 encoding to fix issue
    # Missing values or spaces would be replaced by np.nan
    #I have downloaded the data and will use local file (So Much Data Candy, Seriously (Ng, 2017))
    na_values = [""," ",None,"Missing","NA"]
    df=pd.read_csv(filename,encoding='latin-1',na_values=na_values)
    df.head()
    return df

#To perform 2 tasks out of below list Chapter 7
# 1. Filter out missing data
# 2. Fill in missing data
# 3. Remove duplicates
# 4. Transform data using either mapping or a function
# 5. Replace values
# 6. Discretization and Binning
# 7. Manipulate Strings
# 8. Noticed that missing Values are automatically populated as NaN

def ExerciseChapter_7(df):
    df.set_index(['Internal ID'],inplace=True)
    df.head()

    # It shows that around 21 rows dropped as it didn't had any value populated.
    # Check if there are any duplicate records.
    print(df.duplicated().sum())

    df.drop_duplicates(inplace=True)
    df.shape
    #print(df)

    # There are few columns for which more than 50% records have missing values i.e. NaN.
    # I don't think those columns would be of any use, hence dropping those columns.
    # # Dropping columns where more than 50% records are NaN
    df.dropna(thresh=df.shape[0]*0.5,axis=1,inplace=True)
    df.shape

    # Dropping rows where more than 50% records are NaN
    df.dropna(thresh=df.shape[1]*0.5,axis=0,inplace=True)
    df.shape
    #print(df)

    # Country names are missing at some places and also in different cases like US, us and some places usa
    df['Q4: COUNTRY'].unique()
    # There are too many 129 countries with varying names. I will just try to correct country name for US.
    # First changing all contry names to uppercase.
    df['Q4: COUNTRY'] = df['Q4: COUNTRY'].str.upper()
    df['Q4: COUNTRY'].unique()
    print("df['Q4: COUNTRY'].unique().size Value :", df['Q4: COUNTRY'].unique().size)

    usa_names_map = {
    'USA': 'USA',
    'USA ': 'USA',
    'US': 'USA',
    'UNITED STATES': 'USA',
    'U.S.': 'USA',
    'UNITED STATES OF AMERICA': 'USA',
    'US OF A': 'USA',
    'U.S.A.': 'USA',
    'USAUSAUSA': 'USA',
    'THE UNITED STATES':'USA',
    'UNIED STATES': 'USA',
    'USA! USA! USA!':'USA',
    'USAA': 'USA',
    'AMERICA': 'USA',
    'UNHINGED STATES': 'USA',
    'THE UNITED STATES OF AMERICA': 'USA', 
    'UNITE STATES': 'USA',
    'UNITED SATES': 'USA',
    'UNITES STATES': 'USA',
    'UNITED STSTES': 'USA',
    'UNITED STATES ': 'USA',
    'UNITED STATES OF AMERICA ': 'USA',
    'U S': 'USA',
    'UNITED STATED': 'USA',
    'USA USA USA!!!!': 'USA' }
    df['Q4: COUNTRY'] = df['Q4: COUNTRY'].map(usa_names_map).fillna(df['Q4: COUNTRY'])
    df['Q4: COUNTRY'].unique()
 
    print("df['Q4: COUNTRY'].unique().size Value :",df['Q4: COUNTRY'].unique().size)

    return df

# To perform below 2 tasks Chapter 8
# 1. Create hierarchical index
# 2. Reshape
def ExcerciseChapter_8(df):
    candy_names = {x:x.split('Q6 | ')[1] for x in df.columns if x.startswith('Q6 | ')}
    
    #candy_names = [ {x:x.split('Q6 | ')[1]} for x in df.columns if x.startswith('Q6 | ')] 
    # #candy_names
    df.rename(columns=candy_names,inplace=True)
    df.reset_index(inplace=True)
    #df.set_index(list(candy_names.values()),inplace=True)
    #df.head()

    df.dropna(subset=['Q2: GENDER','Q3: AGE'],inplace=True)
    df.shape

    # Created Hierachial Data using columns Gender and Age
    df.set_index(['Q2: GENDER','Q3: AGE'],inplace=True)
    df.head()

    new_index = pd.Index(list(candy_names.values()),name='candy')
    new_df = df.reindex(columns=new_index)
    new_df.head()

    #print('new_df', new_df)

    ldata = new_df.stack().reset_index().rename(columns={0: 'candy_liking'})
    ldata[:10]
   
    yield df
    yield ldata


# To perform below 2 tasks Chapter 10
# 1. Grouping with Dicts/Series
# 2. Cross Tabs
def ExcerciseChapter_10(df, ldata):
    df.reset_index(inplace=True)
    df.head()
    df.shape

    print("ldata Value :",ldata)
    print("df2 Value :",df)

    # dropping records where either country or State/Province/City field have missing data
    df.dropna(subset=['Q4: COUNTRY','Q5: STATE, PROVINCE, COUNTY, ETC'],inplace=True)
    df.shape

    mapping = {
        'Q4: COUNTRY': 'COUNTRY',
        'Q5: STATE, PROVINCE, COUNTY, ETC': 'LOCATION'
    }

    by_column = df.groupby(mapping, axis=1)
    print('by_column Value :', by_column)
    
    print("by_column.describe() Value :", by_column.describe())

    # Now I would like to find count of each candy marked as JOY , MEH and DISPAIR. I will use CrossTab function.
    # I also want to find most liked Candy.
    # For this analysis , I will use dataframe created in Chapter 8 Exercise i.e. "ldata"
    ldata[:10]
    
    # Now Running crosstab function over candy and candy_liking column.
    pd.crosstab([ldata['candy']],ldata.candy_liking,margins=True)
    crosstab_df = pd.crosstab([ldata['candy']],ldata.candy_liking)
    print(crosstab_df[crosstab_df['JOY'] == crosstab_df['JOY'].max()])

# To perform below 2 tasks Chapter 11
# 1. Convert between string and date time
# 2. Convert timestamps to periods and back
def ExcerciseChapter_11():
    na_values = [""," ",None,"Missing","NA"]
    df_2016= pd.read_excel("BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx",na_values=na_values)
    df_2016.head()
    print("df_2016 Value :",df_2016)

    # Checking Data type of Timestamp column.
    dataTypeObj = df_2016.dtypes['Timestamp']
    print("dataTypeObj Value :",dataTypeObj)

    day_of_week = df_2016['Timestamp'].apply(lambda x: x.strftime('%A'))
    # Adding day_of_week as 2nd column in dataframe
    df_2016.insert(loc=1,column='Day_of_Week',value=day_of_week)
    df_2016.sample(5)

    print(f" Data collected is between date range {df_2016['Timestamp'].min()} - {df_2016['Timestamp'].max()}")

    # Convert timestamps to periods and back
    session=pd.cut(df_2016.Timestamp.dt.hour,
               [0,6,12,18,23],
               labels=['Night','Morning','Afternoon','Evening'],
               include_lowest=True)
    print("session.value_counts() Value:", session.value_counts())
    
def main():
    print("Inside Main function")
    # function to read the So Much Data Candy, Seriously (Ng, 2017) data
    df = ReadData("candyhierarchy2017.csv")
    print("df.shape Value:", df.shape)
    print("df.isna().sum() Value:", df.isna().sum())

    # Perform 2 tasks out of below list Chapter 7
    df = ExerciseChapter_7(df)

    
    # Perform 2 tasks out of below list Chapter 8
    result  = ExcerciseChapter_8(df)

    df = next(result)
    ldata = next(result)
    print("next(result).1 Value:",df)
    print("next(result).2 Value :",ldata)

    # Perform 2 tasks out of below list Chapter 10
    ExcerciseChapter_10(df, ldata)
    
    # Perform 2 tasks out of below list Chapter 11
    ExcerciseChapter_11()

if __name__ == "__main__":
    main()

Inside Main function
(2460, 120)
Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64
35
79
61
next(result).1                     Internal ID Q1: GOING OUT? Q4: COUNTRY  \
Q2: GENDER Q3: AGE                                           
Male       44          90272821             No         USA   
           40          90272840             No         USA   
           23          90272841             No         USA   
           33          90272854             No      CANADA   
           40          90272858             No      CANADA   
...                         ...            ...         ...   
Female     26          90314022             No         USA